In [ ]:
import pandas as pd
import plotly.express as px

# Load your dataset
df = pd.read_excel("/content/Data-2022-2024.xlsx")

# Convert 'ReportDate' to datetime and create 'Month_Year' string for animation
df['ReportDate'] = pd.to_datetime(df['ReportDate'])
df['Month_Year'] = df['ReportDate'].dt.strftime('%Y-%m')

# Calculate total flared volume in Bakken region (2022–2024)
total_flared_mcf = df['Flared'].sum()
total_flared_mcf = round(total_flared_mcf, 2)

# Set fixed color scale bounds for consistent visual comparison
zmin = 0
zmax = 65000  # Adjust based on your dataset if needed

# Create animated density heatmap
fig = px.density_mapbox(df,
                        lat='Lat',
                        lon='Long',
                        z='Flared',
                        radius=10,
                        animation_frame="Month_Year",
                        center={"lat": 47.5, "lon": -103.5},  # Center over Bakken
                        zoom=7,
                        mapbox_style="open-street-map",
                        color_continuous_scale="Plasma",
                        range_color=[zmin, zmax],
                        title=f"Total Volume Flared in Bakken (2022–2024): {total_flared_mcf:,.0f} MCF")

fig.update_layout(margin={"r": 0, "t": 40, "l": 0, "b": 0})
fig.show()

In [ ]:
import pandas as pd
import numpy as np
from sklearn.cluster import DBSCAN
import plotly.express as px

# Load data
df = pd.read_excel("/content/Data-2022-2024.xlsx")

# Step 1: Cluster wells within 30 meters using DBSCAN
def cluster_wells(df, distance_threshold=30):
    coords = df[['Lat', 'Long']].to_numpy()
    kms_per_radian = 6371.0088
    epsilon = distance_threshold / 1000
    clustering = DBSCAN(eps=epsilon / kms_per_radian, min_samples=1, metric='haversine').fit(np.radians(coords))
    df['RawCluster'] = clustering.labels_

    # Count number of wells in each cluster
    cluster_sizes = df.groupby('RawCluster')['API_WELLNO'].nunique().reset_index()
    cluster_sizes.columns = ['RawCluster', 'NumWells']

    # Assign Wellpad IDs
    cluster_sizes = cluster_sizes.sort_values(by='NumWells', ascending=False).reset_index(drop=True)
    cluster_sizes['Wellpad'] = cluster_sizes.index + 1

    # Merge back to original df
    df = df.merge(cluster_sizes, on='RawCluster', how='left')

    return df

df = cluster_wells(df)

# Step 2: Generate wellpad category labels with correct order
df['WellpadSizeCategory'] = df['NumWells'].apply(lambda n: f"{n} well" if n == 1 else f"{n} wells")

# Get sorted unique well counts
unique_well_counts = sorted(df['NumWells'].unique())

# Create the ordered category list
category_order = [f"{n} well" if n == 1 else f"{n} wells" for n in unique_well_counts]

# Make WellpadSizeCategory a categorical type with the defined order
df['WellpadSizeCategory'] = pd.Categorical(df['WellpadSizeCategory'], categories=category_order, ordered=True)

# Debugging: Print the categories
print("Categorical Order:", df['WellpadSizeCategory'].cat.categories)
print("Unique Wellpad Size Categories:", df['WellpadSizeCategory'].unique())

# Step 3: Add YearMonth for animation
df['YearMonth'] = pd.to_datetime(df['ReportDate']).dt.to_period('M').astype(str)

# Step 4: Plot
min_flared = df['Flared'].min()
max_flared = df['Flared'].max()

fig = px.scatter_mapbox(df,
                        lat='Lat',
                        lon='Long',
                        color='WellpadSizeCategory',
                        size='Flared',
                        animation_frame='YearMonth',
                        hover_name='WellName',
                        hover_data={
                            'API_WELLNO': True,
                            'Flared': True,
                            'Company': True,
                            'Wellpad': True,
                            'NumWells': True
                        },
                        category_orders={'WellpadSizeCategory': category_order}, # Explicitly set category order here
                        size_max=60,
                        title="Bakken Wellpads Clustered Within 30m (2022–2024)")

fig.update_layout(
    mapbox_style="open-street-map",
    mapbox=dict(center=dict(lat=df['Lat'].mean(), lon=df['Long'].mean()), zoom=8),
    legend_title_text="Wells per Wellpad",
    margin={"r":0, "t":40, "l":0, "b":0},
    coloraxis_colorbar=None,
    annotations=[
        dict(
            x=1,
            y=0,
            xref="paper",
            yref="paper",
            text=f"Flaring Volume:<br>Smallest Bubble: ~{min_flared:.0f} MCF<br>Largest Bubble: ~{max_flared:.0f} MCF",
            showarrow=False,
            align="right",
            bgcolor="rgba(255, 255, 255, 0.8)",
            bordercolor="black",
            borderwidth=1,
            borderpad=4,
        )
    ]
)


fig.show()

In [ ]:
import plotly.io as pio
from google.colab import files

# Save the Plotly figure as an HTML file
pio.write_html(fig, "Clustered_wells_Bakken_F.html", include_plotlyjs="cdn")

# Give a small delay before downloading (sometimes needed in Colab)
import time
time.sleep(2)

# Download the file
files.download("Clustered_wells_Bakken_F.html")

In [ ]:
import pandas as pd
import numpy as np
from sklearn.cluster import DBSCAN
import plotly.express as px

# Load data
df = pd.read_excel("/content/Data-2022-2024.xlsx")

# Step 1: Cluster wells within 30 meters using DBSCAN
def cluster_wells(df, distance_threshold=30):
    coords = df[['Lat', 'Long']].to_numpy()
    kms_per_radian = 6371.0088
    epsilon = distance_threshold / 1000
    clustering = DBSCAN(eps=epsilon / kms_per_radian, min_samples=1, metric='haversine').fit(np.radians(coords))
    df['RawCluster'] = clustering.labels_

    # Count number of wells in each cluster
    cluster_sizes = df.groupby('RawCluster')['API_WELLNO'].nunique().reset_index()
    cluster_sizes.columns = ['RawCluster', 'NumWells']

    # Assign Wellpad IDs
    cluster_sizes = cluster_sizes.sort_values(by='NumWells', ascending=False).reset_index(drop=True)
    cluster_sizes['Wellpad'] = cluster_sizes.index + 1

    # Merge back to original df
    df = df.merge(cluster_sizes, on='RawCluster', how='left')

    return df

df = cluster_wells(df)

# Step 2: Generate wellpad category labels with correct order
df['WellpadSizeCategory'] = df['NumWells'].apply(lambda n: f"{n} well" if n == 1 else f"{n} wells")

# Get sorted unique well counts
unique_well_counts = sorted(df['NumWells'].unique())

# Create the ordered category list
category_order = [f"{n} well" if n == 1 else f"{n} wells" for n in unique_well_counts]

# Make WellpadSizeCategory a categorical type with the defined order
df['WellpadSizeCategory'] = pd.Categorical(df['WellpadSizeCategory'], categories=category_order, ordered=True)

# Debugging: Print the categories
print("Categorical Order:", df['WellpadSizeCategory'].cat.categories)
print("Unique Wellpad Size Categories:", df['WellpadSizeCategory'].unique())

# Step 3.1: Add YearMonth (if not already present)
df['YearMonth'] = pd.to_datetime(df['ReportDate']).dt.to_period('M').astype(str)

# Step 3.2: Aggregate flaring per wellpad per month
monthly_flared = df.groupby(['Wellpad', 'YearMonth']).agg({
    'Flared': 'sum',
    'Lat': 'first',
    'Long': 'first',
    'WellpadSizeCategory': 'first',
    'NumWells': 'first'
}).reset_index()

# Step 3.3: Filter for Scenario 1 (≥1896 MCF/month)
scenario1_df = monthly_flared[monthly_flared['Flared'] >= 1896]

# Step 3.4: Filter for Scenario 2 (≥7584 MCF/month)
scenario2_df = monthly_flared[monthly_flared['Flared'] >= 7584]

# Count unique wellpads in each scenario
count_1896 = scenario1_df['Wellpad'].nunique()
count_7584 = scenario2_df['Wellpad'].nunique()

# Get wellpad size category ordering
category_order = sorted(df['WellpadSizeCategory'].unique(), key=lambda x: int(x.split()[0]))
scenario1_df['WellpadSizeCategory'] = pd.Categorical(scenario1_df['WellpadSizeCategory'],
                                                     categories=category_order,
                                                     ordered=True)
scenario2_df['WellpadSizeCategory'] = pd.Categorical(scenario2_df['WellpadSizeCategory'],
                                                     categories=category_order,
                                                     ordered=True)

# Scenario 1 - Plotly Map
fig1 = px.scatter_mapbox(
    scenario1_df,
    lat='Lat',
    lon='Long',
    color='WellpadSizeCategory',
    size='Flared',
    animation_frame='YearMonth',
    hover_name='Wellpad',
    hover_data={
        'Flared': True,
        'NumWells': True,
        'YearMonth': True
    },
    category_orders={'WellpadSizeCategory': category_order},
    size_max=60,
    title=f"Figure 1: Wellpads Flaring ≥1896 MCF/Month (2022–2024) — {count_1896} Wellpads"
)

fig1.update_layout(
    mapbox_style="open-street-map",
    mapbox=dict(center=dict(lat=df['Lat'].mean(), lon=df['Long'].mean()), zoom=8),
    legend_title_text="Wells per Wellpad",
    margin={"r": 0, "t": 40, "l": 0, "b": 0},
    coloraxis_colorbar=None
)

# Scenario 2 - Plotly Map
fig2 = px.scatter_mapbox(
    scenario2_df,
    lat='Lat',
    lon='Long',
    color='WellpadSizeCategory',
    size='Flared',
    animation_frame='YearMonth',
    hover_name='Wellpad',
    hover_data={
        'Flared': True,
        'NumWells': True,
        'YearMonth': True
    },
    category_orders={'WellpadSizeCategory': category_order},
    size_max=60,
    title=f"Figure 2: Wellpads Flaring ≥7584 MCF/Month (2022–2024) — {count_7584} Wellpads"
)

fig2.update_layout(
    mapbox_style="open-street-map",
    mapbox=dict(center=dict(lat=df['Lat'].mean(), lon=df['Long'].mean()), zoom=8),
    legend_title_text="Wells per Wellpad",
    margin={"r": 0, "t": 40, "l": 0, "b": 0},
    coloraxis_colorbar=None
)

# Show figures
fig1.show()
fig2.show()


In [ ]:
# Save Scenario 1 plot to HTML
fig1.write_html("Wellpads_Flaring_Above_1896_MCF_Per_Month_2022_2024.html")

# Save Scenario 2 plot to HTML
fig2.write_html("Wellpads_Flaring_Above_7584_MCF_Per_Month_2022_2024.html")

In [ ]:
# Define columns you want in the export
export_columns = [
    'ReportDate', 'API_WELLNO', 'FileNo', 'Company', 'WellName',
    'Section', 'Township', 'Range', 'Gas', 'GasSold', 'Flared',
    'Lat', 'Long', 'Wellpad', 'YearMonth', 'RawCluster',
    'WellpadSizeCategory', 'NumWells'
]

# Scenario 1 CSV
scenario1_export = df[df['Wellpad'].isin(scenario1_df['Wellpad'])][export_columns]
scenario1_export.rename(columns={'RawCluster': 'OrderedWellpadID'}, inplace=True)
scenario1_export.to_csv('scenario1_wellpads_flared_1896plus.csv', index=False)

# Scenario 2 CSV
scenario2_export = df[df['Wellpad'].isin(scenario2_df['Wellpad'])][export_columns]
scenario2_export.rename(columns={'RawCluster': 'OrderedWellpadID'}, inplace=True)
scenario2_export.to_csv('scenario2_wellpads_flared_7584plus.csv', index=False)


In [ ]:
import pandas as pd

# Replace the filenames below with the correct ones from your Colab environment
scenario1_df = pd.read_csv("/content/scenario1_wellpads_flared_1896plus.csv")
scenario2_df = pd.read_csv("/content/scenario2_wellpads_flared_7584plus.csv")

# Proceed with the same analysis and categorization function
def categorize_flare_frequency(df, threshold):
    df['YearMonth'] = pd.to_datetime(df['YearMonth']).dt.to_period('M')
    grouped = df.groupby(['OrderedWellpadID', 'YearMonth'])['Flared'].sum().reset_index()
    total_months = grouped.groupby('OrderedWellpadID')['YearMonth'].count()
    high_flaring_months = grouped[grouped['Flared'] >= threshold].groupby('OrderedWellpadID')['YearMonth'].count()
    high_flaring_months = total_months.to_frame().join(high_flaring_months, how='left', rsuffix='_high').fillna(0)['YearMonth_high']
    flare_percentage = (high_flaring_months / total_months * 100).fillna(0)

    def categorize(p):
        if p >= 90:
            return "90–100% months"
        elif p >= 75:
            return "75–90% months"
        elif p >= 50:
            return "50–75% months"
        elif p >= 25:
            return "25–50% months"
        else:
            return "Below 25% months"

    flare_category = flare_percentage.apply(categorize)
    flare_breaks = (total_months - high_flaring_months).fillna(0)
    flare_rank = flare_breaks.rank(method='dense', ascending=True).astype(int)
    df['FlareCategory'] = df['OrderedWellpadID'].map(flare_category)
    df['FlareRank'] = df['OrderedWellpadID'].map(flare_rank)
    category_counts = flare_category.value_counts().to_dict()
    return df, category_counts


In [ ]:
# Calculate total number of months each wellpad appears in the dataset
total_months_1 = scenario1_df.groupby('OrderedWellpadID')['YearMonth'].nunique()

# Calculate number of months each wellpad flared ≥ 1896 MCF
high_flare_months_1 = scenario1_df[scenario1_df['Flared'] >= 1896].groupby('OrderedWellpadID')['YearMonth'].nunique()

# Compute percentage of high-flaring months
flare_percentage_1 = (high_flare_months_1 / total_months_1 * 100).fillna(0)

# Categorize each wellpad based on high-flaring percentage
def categorize_flare_percentage(p):
    if p >= 90:
        return "90–100% months"
    elif p >= 75:
        return "75–90% months"
    elif p >= 50:
        return "50–75% months"
    elif p >= 25:
        return "25–50% months"
    else:
        return "<25% months"

flare_category_1 = flare_percentage_1.apply(categorize_flare_percentage)

# Calculate flare breaks and ranks
flare_breaks_1 = (total_months_1 - high_flare_months_1).fillna(0)
flare_rank_1 = flare_breaks_1.rank(method='dense', ascending=True).astype(int)

# Create wellpad-level summary DataFrame
wellpad_summary_1 = scenario1_df.groupby('OrderedWellpadID').first().reset_index()
wellpad_summary_1 = wellpad_summary_1[['OrderedWellpadID', 'Lat', 'Long', 'Company', 'WellName']]
wellpad_summary_1['FlareCategory'] = wellpad_summary_1['OrderedWellpadID'].map(flare_category_1)
wellpad_summary_1['FlareRank'] = wellpad_summary_1['OrderedWellpadID'].map(flare_rank_1)

In [ ]:
# Total number of months per wellpad
total_months_2 = scenario2_df.groupby('OrderedWellpadID')['YearMonth'].nunique()

# High-flaring months ≥ 7584 MCF
high_flare_months_2 = scenario2_df[scenario2_df['Flared'] >= 7584].groupby('OrderedWellpadID')['YearMonth'].nunique()

# Percentage calculation
flare_percentage_2 = (high_flare_months_2 / total_months_2 * 100).fillna(0)

# Categorization
flare_category_2 = flare_percentage_2.apply(categorize_flare_percentage)

# Breaks and rank
flare_breaks_2 = (total_months_2 - high_flare_months_2).fillna(0)
flare_rank_2 = flare_breaks_2.rank(method='dense', ascending=True).astype(int)

# Summary DataFrame
wellpad_summary_2 = scenario2_df.groupby('OrderedWellpadID').first().reset_index()
wellpad_summary_2 = wellpad_summary_2[['OrderedWellpadID', 'Lat', 'Long', 'Company', 'WellName']]
wellpad_summary_2['FlareCategory'] = wellpad_summary_2['OrderedWellpadID'].map(flare_category_2)
wellpad_summary_2['FlareRank'] = wellpad_summary_2['OrderedWellpadID'].map(flare_rank_2)

In [ ]:
import plotly.express as px

# Plot for Scenario 1 (≥1896 MCF/month)
fig1 = px.scatter_mapbox(
    wellpad_summary_1,
    lat='Lat',
    lon='Long',
    color='FlareCategory',
    size=[10]*len(wellpad_summary_1),
    hover_name='WellName',
    hover_data={'OrderedWellpadID': True, 'Company': True, 'FlareCategory': True, 'FlareRank': True},
    title="Wellpads ≥1896 MCF/Month (Categorized by % of High Flaring Months)",
    zoom=8
)
fig1.update_layout(
    mapbox_style="open-street-map",
    mapbox_center={"lat": wellpad_summary_1['Lat'].mean(), "lon": wellpad_summary_1['Long'].mean()},
    margin={"r":0, "t":40, "l":0, "b":0}
)
fig1.show()
fig1.write_html("Wellpads_1896MCF_Categorized.html")
scenario1_df.to_csv("Wellpads_1896MCF_Categorized.csv", index=False)

# Plot for Scenario 2 (≥7584 MCF/month)
fig2 = px.scatter_mapbox(
    wellpad_summary_2,
    lat='Lat',
    lon='Long',
    color='FlareCategory',
    size=[10]*len(wellpad_summary_2),
    hover_name='WellName',
    hover_data={'OrderedWellpadID': True, 'Company': True, 'FlareCategory': True, 'FlareRank': True},
    title="Wellpads ≥7584 MCF/Month (Categorized by % of High Flaring Months)",
    zoom=8
)
fig2.update_layout(
    mapbox_style="open-street-map",
    mapbox_center={"lat": wellpad_summary_2['Lat'].mean(), "lon": wellpad_summary_2['Long'].mean()},
    margin={"r":0, "t":40, "l":0, "b":0}
)
fig2.show()
fig2.write_html("Wellpads_7584MCF_Categorized.html")
scenario2_df.to_csv("Wellpads_7584MCF_Categorized.csv", index=False)

In [ ]:
def generate_company_summary(df_raw, summary_df, threshold, scenario_name):
    # Merge raw flaring data with categorized wellpad summary
    merged_df = df_raw.merge(summary_df[['OrderedWellpadID', 'FlareCategory']], on='OrderedWellpadID', how='left')

    # Filter for wellpads meeting the monthly flaring threshold
    merged_df = merged_df[merged_df['Flared'] >= threshold]

    # Calculate total flared gas per wellpad
    total_flared_per_wellpad = merged_df.groupby(['OrderedWellpadID'])['Flared'].sum()

    # Map flare totals back to summary_df
    summary_df = summary_df.copy()
    summary_df['TotalFlaredMCF'] = summary_df['OrderedWellpadID'].map(total_flared_per_wellpad).fillna(0)

    # Group by Company and FlareCategory, count clusters and sum flared gas
    grouped = summary_df.groupby(['Company', 'FlareCategory']).agg(
        Clusters=('OrderedWellpadID', 'count'),
        TotalFlaredMCF=('TotalFlaredMCF', 'sum')
    ).reset_index()

    # Pivot to get category columns
    pivot_df = grouped.pivot(index='Company', columns='FlareCategory', values='Clusters').fillna(0).astype(int)
    pivot_df = pivot_df[['90–100% months', '75–90% months', '50–75% months', '25–50% months']].fillna(0).astype(int)  # Ensure column order

    # Sum flaring across all categories per company
    total_flared = summary_df.groupby('Company')['TotalFlaredMCF'].sum()

    # Compute estimated revenue
    pivot_df['Estimated Revenue ($)'] = total_flared * 13.374
    pivot_df = pivot_df.reset_index()

    # Save to CSV
    file_name = f"{scenario_name.replace(' ', '_').replace('≥', 'ge')}_Company_Summary.csv"
    pivot_df.to_csv(file_name, index=False)
    print(f"✅ Saved: {file_name}")
    return pivot_df

In [ ]:
# Scenario 1
company_summary_1 = generate_company_summary(
    df_raw=scenario1_df,
    summary_df=wellpad_summary_1,
    threshold=1896,
    scenario_name="Wellpads ≥1896 MCF"
)

# Scenario 2
company_summary_2 = generate_company_summary(
    df_raw=scenario2_df,
    summary_df=wellpad_summary_2,
    threshold=7584,
    scenario_name="Wellpads ≥7584 MCF"
)

In [ ]:
import pandas as pd
import numpy as np
import math

def calculate_units_revenue_from_wellpads(df):
    df.columns = df.columns.str.strip()  # Strip column names
    df['Flared'] = pd.to_numeric(df['Flared'], errors='coerce')
    df = df.dropna(subset=['Company', 'Flared', 'OrderedWellpadID', 'YearMonth'])

    # Total months per wellpad
    total_months = df.groupby('OrderedWellpadID')['YearMonth'].nunique()

    # Count months with flaring ≥1896
    high_flare_months = df[df['Flared'] >= 1896].groupby('OrderedWellpadID')['YearMonth'].nunique()

    # Calculate percent of months each wellpad had high flaring
    percent_active = (high_flare_months / total_months).fillna(0)

    # Categorize wellpads
    def categorize(pct):
        if pct >= 0.9:
            return '90-100% months'
        elif pct >= 0.75:
            return '75-90% months'
        elif pct >= 0.5:
            return '50-75% months'
        elif pct >= 0.25:
            return '25-50% months'
        else:
            return None

    wellpad_categories = percent_active.apply(categorize)
    df['Category'] = df['OrderedWellpadID'].map(wellpad_categories)

    # Remove uncategorized wellpads
    df = df[df['Category'].notna()]

    categories = ['90-100% months', '75-90% months', '50-75% months', '25-50% months']
    results = []

    for company in df['Company'].unique():
        row = {'Company': company}
        total_revenue = 0

        for cat in categories:
            sub_df = df[(df['Company'] == company) & (df['Category'] == cat)]
            wellpads = sub_df['OrderedWellpadID'].unique()

            wellpad_units = 0
            wellpad_revenue = 0.0

            for wellpad_id in wellpads:
                wp_df = sub_df[(sub_df['OrderedWellpadID'] == wellpad_id) & (sub_df['Flared'] >= 1896)]

                if not wp_df.empty:
                    avg_flared = wp_df['Flared'].mean()
                    units_needed = int(avg_flared / 1896)  # Only count fully used units
                    revenue = avg_flared * 13.374 if units_needed > 0 else 0.0

                    wellpad_units += units_needed
                    wellpad_revenue += revenue

            row[cat] = len(wellpads)
            row[f'Units_{cat}'] = wellpad_units
            row[f'Revenue_{cat}'] = round(wellpad_revenue, 2)
            total_revenue += wellpad_revenue

        row['Final Cumulative Estimated Revenue ($)'] = round(total_revenue, 2)
        results.append(row)

    result_df = pd.DataFrame(results)

    # Reorder columns
    cols = ['Company']
    for cat in categories:
        cols.extend([cat, f'Units_{cat}', f'Revenue_{cat}'])
    cols.append('Final Cumulative Estimated Revenue ($)')
    return result_df[cols]


In [ ]:
# Load your wellpad-level dataset
df_1896 = pd.read_csv("/content/scenario1_wellpads_flared_1896plus.csv")

# Run the analysis
result_df = calculate_units_revenue_from_wellpads(df_1896)

# Save result
result_df.to_csv("Scenario1_Units_Revenue_Per_Company.csv", index=False)
print("✅ Scenario 1 saved.")


In [ ]:
import pandas as pd
import numpy as np
import math

def calculate_units_revenue_from_wellpads(df):
    df.columns = df.columns.str.strip()  # Strip column names
    df['Flared'] = pd.to_numeric(df['Flared'], errors='coerce')
    df = df.dropna(subset=['Company', 'Flared', 'OrderedWellpadID', 'YearMonth'])

    # Total months per wellpad
    total_months = df.groupby('OrderedWellpadID')['YearMonth'].nunique()

    # Count months with flaring ≥7584
    high_flare_months = df[df['Flared'] >= 7584].groupby('OrderedWellpadID')['YearMonth'].nunique()

    # Calculate percent of months each wellpad had high flaring
    percent_active = (high_flare_months / total_months).fillna(0)

    # Categorize wellpads
    def categorize(pct):
        if pct >= 0.9:
            return '90-100% months'
        elif pct >= 0.75:
            return '75-90% months'
        elif pct >= 0.5:
            return '50-75% months'
        elif pct >= 0.25:
            return '25-50% months'
        else:
            return None

    wellpad_categories = percent_active.apply(categorize)
    df['Category'] = df['OrderedWellpadID'].map(wellpad_categories)

    # Remove uncategorized wellpads
    df = df[df['Category'].notna()]

    categories = ['90-100% months', '75-90% months', '50-75% months', '25-50% months']
    results = []

    for company in df['Company'].unique():
        row = {'Company': company}
        total_revenue = 0

        for cat in categories:
            sub_df = df[(df['Company'] == company) & (df['Category'] == cat)]
            wellpads = sub_df['OrderedWellpadID'].unique()

            wellpad_units = 0
            wellpad_revenue = 0.0

            for wellpad_id in wellpads:
                wp_df = sub_df[(sub_df['OrderedWellpadID'] == wellpad_id) & (sub_df['Flared'] >= 7584)]

                if not wp_df.empty:
                    avg_flared = wp_df['Flared'].mean()
                    units_needed = int(avg_flared / 7584)  # Only count fully used units
                    revenue = avg_flared * 13.374 if units_needed > 0 else 0.0

                    wellpad_units += units_needed
                    wellpad_revenue += revenue

            row[cat] = len(wellpads)
            row[f'Units_{cat}'] = wellpad_units
            row[f'Revenue_{cat}'] = round(wellpad_revenue, 2)
            total_revenue += wellpad_revenue

        row['Final Cumulative Estimated Revenue ($)'] = round(total_revenue, 2)
        results.append(row)

    result_df = pd.DataFrame(results)

    # Reorder columns
    cols = ['Company']
    for cat in categories:
        cols.extend([cat, f'Units_{cat}', f'Revenue_{cat}'])
    cols.append('Final Cumulative Estimated Revenue ($)')
    return result_df[cols]

In [ ]:
# Load your wellpad-level dataset
df_7584 = pd.read_csv("/content/scenario2_wellpads_flared_7584plus.csv")

# Run the analysis
result_df = calculate_units_revenue_from_wellpads(df_7584)

# Save result
result_df.to_csv("Scenario2_Units_Revenue_Per_Company.csv", index=False)
print("✅ Scenario 2 saved.")